# Module 1 Homework: Docker & SQL

In this homework we'll prepare the environment and practice
Docker and SQL

When submitting your homework, you will also need to include
a link to your GitHub repository or other public code-hosting
site.

This repository should contain the code for solving the homework.

When your solution has SQL or shell commands and not code
(e.g. python files) file format, include them directly in
the README file of your repository.


## Question 1. Understanding Docker images

Run docker with the `python:3.13` image. Use an entrypoint `bash` to interact with the container.

What's the version of `pip` in the image?

- 25.3
- 24.3.1
- 24.2.1
- 23.3.1

**Answer: 25.3**

docker run -it --entrypoint=bash python:3.13

pip --version



## Question 2. Understanding Docker networking and docker-compose

Given the following `docker-compose.yaml`, what is the `hostname` and `port` that pgadmin should use to connect to the postgres database?

```yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data
```

- postgres:5433
- localhost:5432
- db:5433
- postgres:5432
- db:5432

If multiple answers are correct, select any 

**Answer: db:5432**



## Prepare the Data

Download the green taxi trips data for November 2025:

```bash
wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
```

You will also need the dataset with zones:

```bash
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
```


In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('green_tripdata_2025-11.parquet')

In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 46912 entries, 0 to 46911
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               46912 non-null  int32         
 1   lpep_pickup_datetime   46912 non-null  datetime64[us]
 2   lpep_dropoff_datetime  46912 non-null  datetime64[us]
 3   store_and_fwd_flag     41343 non-null  str           
 4   RatecodeID             41343 non-null  float64       
 5   PULocationID           46912 non-null  int32         
 6   DOLocationID           46912 non-null  int32         
 7   passenger_count        41343 non-null  float64       
 8   trip_distance          46912 non-null  float64       
 9   fare_amount            46912 non-null  float64       
 10  extra                  46912 non-null  float64       
 11  mta_tax                46912 non-null  float64       
 12  tip_amount             46912 non-null  float64       
 13  tolls_amount

In [4]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [5]:
taxi_zone = pd.read_csv('taxi_zone_lookup.csv')

In [6]:
taxi_zone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [7]:
len(taxi_zone)

265

## Question 3. Counting short trips

For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a `trip_distance` of less than or equal to 1 mile?

- 7,853
- 8,007
- 8,254
- 8,421

**Answer: 8007**


In [8]:
from pandasql import sqldf

In [9]:
# Define your SQL query as a string

q3 = """
SELECT COUNT(*)
FROM df
WHERE
  lpep_pickup_datetime BETWEEN '2025-11-01' AND '2025-12-01'
  AND trip_distance <= 1
"""

In [10]:
# Execute the query and store the result in a new DataFrame
result_df = sqldf(q3, globals()) # or locals()

print(result_df)

   COUNT(*)
0      8007


## Question 4. Longest trip for each day

Which was the pick up day with the longest trip distance? Only consider trips with `trip_distance` less than 100 miles (to exclude data errors).

Use the pick up time for your calculations.

- 2025-11-14
- 2025-11-20
- 2025-11-23
- 2025-11-25

**Answer: 2025-11-14**


In [19]:
# Define your SQL query as a string

q4 = """
SELECT
    DATE(lpep_pickup_datetime) AS pickup_date,
    MAX(trip_distance) AS max_distance
FROM df
WHERE trip_distance < 100
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1
"""

In [20]:
# Execute the query and store the result in a new DataFrame
result_df = sqldf(q4, globals()) # or locals()

print(result_df)

  pickup_date  max_distance
0  2025-11-14         88.03


## Question 5. Biggest pickup zone

Which was the pickup zone with the largest `total_amount` (sum of all trips) on November 18th, 2025?

- East Harlem North
- East Harlem South
- Morningside Heights
- Forest Hills

**Answer: East Harlem North**

In [24]:
# Define your SQL query as a string

q5 = """
SELECT
    tz.Zone,
    SUM(df.total_amount) AS total
FROM df
INNER JOIN taxi_zone tz
    ON df.PULocationID = tz.LocationID
WHERE
    strftime('%Y-%m-%d', lpep_pickup_datetime) = '2025-11-18'
GROUP BY tz.Zone
ORDER BY total DESC
LIMIT 5
"""

In [25]:
# Execute the query and store the result in a new DataFrame
result_df = sqldf(q5, globals()) # or locals()

print(result_df)

                       Zone    total
0         East Harlem North  9281.92
1         East Harlem South  6696.13
2              Central Park  2378.79
3  Washington Heights South  2139.05
4       Morningside Heights  2100.59


## Question 6. Largest tip

For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

Note: it's `tip` , not `trip`. We need the name of the zone, not the ID.

- JFK Airport
- Yorkville West
- East Harlem North
- LaGuardia Airport

**Answer: Yorkville West**


In [129]:
# Define your SQL query as a string

q6 = """
SELECT
    strftime('%Y-%m-%d', lpep_pickup_datetime) AS lpep_pickup_datetime,
    dotz.Zone AS dropoff_zone,
    tip_amount
FROM df
INNER JOIN taxi_zone putz
    ON df.PULocationID = putz.LocationID
INNER JOIN taxi_zone dotz
    ON df.DOLocationID = dotz.LocationID
WHERE
    strftime('%Y-%m-%d', lpep_pickup_datetime) BETWEEN '2025-11-01' AND '2025-11-30'
    AND putz.Zone = 'East Harlem North'
ORDER BY 3 DESC
"""

In [130]:
# Execute the query and store the result in a new DataFrame
result_df = sqldf(q6, globals()) # or locals()

print(result_df)

      lpep_pickup_datetime                   dropoff_zone  tip_amount
0               2025-11-30                 Yorkville West       81.89
1               2025-11-23              LaGuardia Airport       50.00
2               2025-11-29              East Harlem North       45.00
3               2025-11-24  Long Island City/Queens Plaza       34.25
4               2025-11-15                            NaN       28.90
...                    ...                            ...         ...
12036           2025-11-30                 Central Harlem        0.00
12037           2025-11-30                   Bloomingdale        0.00
12038           2025-11-30           Central Harlem North        0.00
12039           2025-11-30                 Yorkville East        0.00
12040           2025-11-30       Washington Heights North        0.00

[12041 rows x 3 columns]


## Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform.
Copy the files from the course repo
[here](../../../01-docker-terraform/terraform/terraform) to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.


## Question 7. Terraform Workflow

Which of the following sequences, respectively, describes the workflow for:
1. Downloading the provider plugins and setting up backend,
2. Generating proposed changes and auto-executing the plan
3. Remove all resources managed by terraform`

Answers:
- terraform import, terraform apply -y, terraform destroy
- teraform init, terraform plan -auto-apply, terraform rm
- terraform init, terraform run -auto-approve, terraform destroy
- terraform init, terraform apply -auto-approve, terraform destroy
- terraform import, terraform apply -y, terraform rm

**Answer: terraform init, terraform apply -auto-approve, terraform destroy**


## Submitting the solutions

* Form for submitting: https://courses.datatalks.club/de-zoomcamp-2026/homework/hw1


## Learning in Public

We encourage everyone to share what they learned. This is called "learning in public".

### Why learn in public?

- Accountability: Sharing your progress creates commitment and motivation to continue
- Feedback: The community can provide valuable suggestions and corrections
- Networking: You'll connect with like-minded people and potential collaborators
- Documentation: Your posts become a learning journal you can reference later
- Opportunities: Employers and clients often discover talent through public learning

You can read more about the benefits [here](https://alexeyondata.substack.com/p/benefits-of-learning-in-public-and).

Don't worry about being perfect. Everyone starts somewhere, and people love following genuine learning journeys!

### Example post for LinkedIn

```
🚀 Week 1 of Data Engineering Zoomcamp by @DataTalksClub complete!

Just finished Module 1 - Docker & Terraform. Learned how to:

✅ Containerize applications with Docker and Docker Compose
✅ Set up PostgreSQL databases and write SQL queries
✅ Build data pipelines to ingest NYC taxi data
✅ Provision cloud infrastructure with Terraform

Here's my homework solution: <LINK>

Following along with this amazing free course - who else is learning data engineering?

You can sign up here: https://github.com/DataTalksClub/data-engineering-zoomcamp/
```

### Example post for Twitter/X


```
🐳 Module 1 of Data Engineering Zoomcamp done!

- Docker containers
- Postgres & SQL
- Terraform & GCP
- NYC taxi data pipeline

My solution: <LINK>

Free course by @DataTalksClub: https://github.com/DataTalksClub/data-engineering-zoomcamp/
```

